<a href="https://colab.research.google.com/github/Sanj-bot/pytorch/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib as plt
%matplotlib inline

In [2]:
transform=transforms.ToTensor()


In [3]:
train_data=datasets.MNIST(root='/cnn_data',train=True,transform=transform, download=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 52898470.07it/s]


Extracting /cnn_data/MNIST/raw/train-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1842592.16it/s]

Extracting /cnn_data/MNIST/raw/train-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 14442317.32it/s]


Extracting /cnn_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4132435.74it/s]

Extracting /cnn_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



In [4]:
test_data=datasets.MNIST(root='/cnn_data',train=False,transform=transform, download=True)

In [5]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
pwd

'/content'

In [7]:
ls

sample_data/


In [8]:
cd ../


/


In [9]:
pwd

'/'

In [10]:
ls

bin@                        datalab/  lib@     mnt/                      root/  tmp/
boot/                       dev/      lib32@   NGC-DL-CONTAINER-LICENSE  run/   tools/
cnn_data/                   etc/      lib64@   opt/                      sbin@  usr/
content/                    home/     libx32@  proc/                     srv/   var/
cuda-keyring_1.0-1_all.deb  kaggle/   media/   python-apt/               sys/


In [11]:
cd cnn_data

/cnn_data


In [12]:
ls

MNIST/


In [13]:
cd MNIST

/cnn_data/MNIST


In [14]:
ls


raw/


In [15]:
cd raw

/cnn_data/MNIST/raw


In [16]:
ls

t10k-images-idx3-ubyte     t10k-labels-idx1-ubyte.gz   train-labels-idx1-ubyte
t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte     train-labels-idx1-ubyte.gz
t10k-labels-idx1-ubyte     train-images-idx3-ubyte.gz


In [17]:
#Convolution and pooling layers

In [18]:
#create a small batch size for images.....
train_loader=DataLoader(train_data,batch_size=10,shuffle=True)

In [19]:
test_loader=DataLoader(train_data,batch_size=10,shuffle=False)

In [20]:
#Define CNN Model
#define cnn layer and what its doing (2 conv layer)
conv1=nn.Conv2d(1,6,3,1)
conv2=nn.Conv2d(6,16,3,1)

In [23]:
#grab 1 MNIST record/image
for i, (X_train,y_train) in enumerate(train_data):
  break

In [24]:
X_train.shape

torch.Size([1, 28, 28])

In [25]:
x=X_train.view(1,1,28,28)

In [26]:
#Perform our first convolution
x=F.relu(conv1(x)) # Rectified linear unit for our activation function

In [27]:
x.shape

torch.Size([1, 6, 26, 26])

In [28]:
#pass thru the pooling layer
x=F.max_pool2d(x,2,2) #kernel of 2 and stride of 2

In [30]:
x.shape

torch.Size([1, 6, 13, 13])

In [31]:
#do our 2nd convolutional layer
x=F.relu(conv2(x))

In [32]:
x.shape
#again we didn't set padding so we loose 2 pixels around the outside of the image

torch.Size([1, 16, 11, 11])

In [33]:
#Padding layer
x=F.max_pool2d(x,2,2)
x.shape # 11/2


torch.Size([1, 16, 5, 5])

In [34]:
#16/19

In [36]:
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(1,6,3,1)
    self.conv2=nn.Conv2d(6,16,3,1)
    #Fully connected layer
    self.fc1=nn.Linear(5*5*16,120)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,10)
  def forward(self,X):
    X=F.relu(self.conv1(X))
    X=F.max_pool2d(X,2,2)
    #2x2 kernal and stride 2
    #Second pass
    X=F.relu(self.conv2(X))
    X=F.max_pool2d(X,2,2)
    #2x2 kernal and stride 2

    #Review to flatten it out
    X=X.view(-1,16*5*5)
    #Negative one so that

    #Fully connected layers
    X=F.relu(self.fc1(X))
    X=F.relu(self.fc2(X))
    X=self.fc3(X)
    return F.log_softmax(X,dim=1)

In [37]:
#create an instance of our model
torch.manual_seed(41)
model=ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [39]:
#LOSS function optimizer
nn.CrossEntropyLoss()
torch.optim.Adam(model.parameters(),lr=0.001)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)